In [1]:
import sagemaker
import os

# 设置相关信息

In [2]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
print(bucket)

sagemaker-us-east-2-069799604450


# 创建SageMaker模型，导入模型脚本，设置超参数等

In [42]:
from sagemaker.tensorflow import TensorFlow
hyperparameters = {'epochs': 1, 'batch_size': 200}
estimator = TensorFlow(entry_point='tf_mnist.py',
                       source_dir='script',
                       train_instance_type='ml.p3.2xlarge',
                       train_instance_count=1,
                       hyperparameters=hyperparameters,
                       role=sagemaker.get_execution_role(),
                       framework_version='1.14',
                       py_version='py3',
                       script_mode=True)

# 训练模型

In [43]:
train_data_dir = 'runqi_tf_script_demo/dataset/train'
estimator.fit({'train':'s3://{}/{}'.format(bucket, train_data_dir)})

2019-08-08 04:03:54 Starting - Starting the training job...
2019-08-08 04:03:56 Starting - Launching requested ML instances......
2019-08-08 04:04:59 Starting - Preparing the instances for training...
2019-08-08 04:05:48 Downloading - Downloading input data...
2019-08-08 04:05:59 Training - Downloading the training image......
2019-08-08 04:07:22 Uploading - Uploading generated training model
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/py

# 部署模型

In [45]:
# 使用Estimator直接部署
predictor = estimator.deploy(initial_instance_count=1,
                             instance_type='ml.p3.2xlarge',
                             endpoint_type='tensorflow-serving')

# 使用模型文件部署
# from sagemaker.tensorflow.serving import Model
# model = Model(model_data='s3://sagemaker-us-east-2-069799604450/sagemaker-tensorflow-scriptmode-2019-08-07-11-02-56-456/output/model.tar.gz', role=sagemaker.get_execution_role())
# predictor = model.deploy(initial_instance_count=1,
#                         instance_type='ml.c5.2xlarge')

W0808 04:12:17.987596 139891432716096 session.py:729] Using already existing model: tensorflow-training-2019-08-08-04-03-53-920


---------------------------------------------------------------------------------------!

# 导入测试数据

In [51]:
import boto3
s3 = boto3.client('s3')
s3.download_file('sagemaker-us-east-2-069799604450', 'runqi_tf_script_demo/dataset/test/test_x.npy', './test_x.npy')
s3.download_file('sagemaker-us-east-2-069799604450', 'runqi_tf_script_demo/dataset/test/test_y.npy', './test_y.npy')
test_x = np.load('./test_x.npy')
test_y = np.load('./test_y.npy')

# 使用endpoint进行预测

In [54]:
predictor.predict(test_x[0:10])

{'predictions': [[0.0,
   0.0,
   1.70789217e-36,
   3.06112975e-26,
   0.0,
   0.0,
   0.0,
   1.0,
   3.56053801e-32,
   2.26501852e-19],
  [0.0,
   3.83586882e-20,
   1.0,
   9.70764268e-14,
   0.0,
   4.68934475e-28,
   1.39734365e-35,
   2.10739617e-37,
   3.55746883e-25,
   0.0],
  [2.64367577e-33,
   1.0,
   1.39873211e-25,
   0.0,
   2.1024979e-37,
   6.48278743e-33,
   1.49968078e-19,
   3.34499308e-18,
   4.30519972e-30,
   1.78129417e-28],
  [0.967329502,
   1.65729332e-07,
   3.72090653e-07,
   0.000131889683,
   6.82859445e-06,
   5.17950184e-06,
   4.65301673e-06,
   8.74194757e-06,
   2.62236881e-06,
   0.0325100608],
  [3.22795256e-17,
   0.0,
   1.33770192e-27,
   6.11282461e-27,
   0.00127253612,
   2.85840054e-32,
   6.37711834e-13,
   1.35222589e-13,
   1.49261821e-24,
   0.9987275],
  [4.75098941e-31,
   1.0,
   1.59767025e-25,
   0.0,
   0.0,
   1.62156727e-33,
   6.27526179e-25,
   1.11803034e-15,
   1.42491898e-29,
   8.89271756e-25],
  [6.04800219e-18,
   3.335